In [ ]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
import random
import xmltodict
from mrcnn.model import log
ROOT_DIR = "./"

In [ ]:
MODEL_DIR = "../model_saved"
COCO_MODEL_PATH = "../mask_rcnn_coco.h5"

# Visualizing Images

In [ ]:
def get_ax(rows = 1, cols = 1, size = 8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize = (size*cols, size*rows))
    return ax

# Configuration of Model 

In [ ]:
class ShapesConfig(Config):
    def __init__(self, num_classes):
        self.NUM_CLASSES = 1 + 5
        super().__init__()
    """Configuration for training.
    """
    # Give the configuration a recognizable name
    NAME = "objects"
    
    
    GPU_COUNT = 1
    IMAGES_PER_GPU = 4

    # Number of classes (including background)
    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 128
    IMAGE_MAX_DIM = 768
    
    DETECTION_MAX_INSTANCES = 20

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 30

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 100

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 5
    
config = ShapesConfig(num_classes = 6)
config.display()

# Creating Dataset

In [ ]:
class ShapesDataset(utils.Dataset):
    """Generates the objects dataset.
    """

    def load_shapes(self , folder_name):
        """Loads all the number of images.
        """
        # Add classes
        self.add_class("objects", 1, "beauty")
        self.add_class("objects", 2, "venus")
        self.add_class("objects", 3, "pharmacy")
        self.add_class("objects" , 4 , "oralcare")
        self.add_class("objects" , 5 , "reject")
        
        i = 1
        
        # Add images
        # Generate random specifications of images (i.e. color and
        # list of shapes sizes and locations). This is more compact than
        # actual images. Images are generated on the fly in load_image().
        for folder in os.listdir(os.path.join(ROOT_DIR , folder_name)):
            for file in os.listdir(os.path.join(ROOT_DIR , folder_name , folder)):
                extension = os.path.splitext(file)[1]
                if extension == ".xml":
                    file = open(os.path.join(ROOT_DIR , folder_name , folder , file))
                    dictionary = xmltodict.parse(file.read())
                    objects = self.image(dict(dictionary))
                    self.add_image("objects" , image_id = i , path = os.path.join(ROOT_DIR , folder_name , folder , dictionary['annotation']['filename']) , width = int(dictionary['annotation']['size']['width']) , height = int(dictionary['annotation']['size']['height']) , depth = int(dictionary['annotation']['size']['depth']) , objects = objects)
                    i = i + 1
    def load_image(self, image_id):
        """Outputs an image from the path of the given image ID.
        Typically this function loads the image from a file
        """
        info = self.image_info[image_id]                         
        path = info['path']
        image = cv2.imread(path)
        return image

    def image_reference(self, image_id):
        """Return the objects data of the image."""
        info = self.image_info[image_id]
        if info["source"] == "objects":
            return info["objects"]
        else:
            super(self.__class__).image_reference(self, image_id)

    def load_mask(self, image_id):
        """Generate instance masks for shapes of the given image ID.
        """
        info = self.image_info[image_id]
        shapes = info['objects']
        count = len(shapes)
        mask = np.zeros([info['height'] , info['width'] , count], dtype = np.uint8)
        for i, (objects , pose , truncated , difficult , dims) in enumerate(info['objects']):
            xmin , ymin , xmax , ymax = dims
            mask[: , : , i:i+1] = cv2.rectangle(mask[: , : , i:i+1].copy() , (xmin, ymin), (xmax, ymax), (255 , 255 , 255) , -1)
        # Handle occlusions
        occlusion = np.logical_not(mask[: , : , -1]).astype(np.uint8)
        for i in range(count-2, -1, -1):
            mask[: , :  , i] = mask[: , :  , i] * occlusion
            occlusion = np.logical_and(occlusion, np.logical_not(mask[: , : , i]))
        # Map class names to class IDs.
        class_ids = np.array([self.class_names.index(s[0]) for s in shapes])
        return mask.astype(np.bool), class_ids.astype(np.int32)

    def objects_values(self, object_data):
        """Outputs specifications of the objects that lies within
        the given image.
        Returns a tuple of five valus:
        * The object name 
        * Object Pose
        * Object Dimensions: A tuple of values that define the shape size
                            and location. Differs per shape type.
        * Object Difficulty
        * Object Truncated or Not
        """
        # Objects
        objects = self.classes(object_data['name'])
        
        # Pose
        pose = object_data['pose']
        
        # Truncated
        truncated = object_data['truncated']
        
        # Difficult
        difficult = object_data['difficult']
        
        # xmin , ymin , xmax , ymax
        xmin = int(object_data['bndbox']['xmin'])
        ymin = int(object_data['bndbox']['ymin'])
        xmax = int(object_data['bndbox']['xmax'])
        ymax = int(object_data['bndbox']['ymax'])
    
        return objects , pose , truncated , difficult ,  (xmin , ymin , xmax , ymax)
    def classes(self , class_name):
        if class_name == "w":
            return "oralcare"
        elif class_name == "orlab":
            return "oralcare"
        elif class_name == "oral":
            return "oralcare"
        elif class_name == "oralb":
            return "oralcare"
        elif class_name == "vicks":
            return "pharmacy"
        elif class_name == "pampers":
            return "pharmacy"
        elif class_name == "whisper":
            return "pharmacy"
        elif class_name == "gillete":
            return "beauty"
        elif class_name == "gillette":
            return "beauty"
        elif class_name == "ambipureVesus":
            return "beauty"
        elif class_name == "ambipur":
            return "beauty"
        elif class_name == "ambipure":
            return "beauty"
        elif class_name == "home":
            return "beauty"
        elif class_name == "ariel":
            return "beauty"
        elif class_name == "olay":
            return "beauty"
        else:
            return class_name

    def image(self, annotation_data):
        """Creates specifications of the image with multiple objects.
        Returns a list of object specifications that can be used to draw the image.
        """
        shapes = []
        boxes = []
        l = 0
        if 'object' in annotation_data['annotation']:
            for i in annotation_data['annotation']['object']:
                if i == "name":
                    l = l + 1
            if l == 0:
                N = len(annotation_data['annotation']['object'])
                for i in range(N):
                    objects , pose , truncated , difficult , dims = self.objects_values(annotation_data['annotation']['object'][i])
                    shapes.append((objects , pose , truncated , difficult , dims))
                    xmin , ymin , xmax , ymax = dims
                    boxes.append([xmin , ymin , xmax , ymax])
            elif l == 1 :
                objects , pose , truncated , difficult , dims = self.objects_values(annotation_data['annotation']['object'])
                shapes.append((objects , pose , truncated , difficult , dims))
                xmin , ymin , xmax , ymax = dims
                boxes.append([xmin , ymin , xmax , ymax])
            # Apply non-max suppression wit 0.3 threshold to avoid
            # shapes covering each other
            if l == 0:
                N = len(annotation_data['annotation']['object'])
                keep_ixs = utils.non_max_suppression(np.array(boxes), np.arange(N), 0.3)
                shapes = [s for i, s in enumerate(shapes) if i in keep_ixs]
        else:
            pass
        return shapes

In [ ]:
dataset_train = ShapesDataset()
dataset_train.load_shapes(folder_name = "train/")
dataset_train.prepare()

dataset_test = ShapesDataset()
dataset_test.load_shapes(folder_name = "test/")
dataset_test.prepare()

## Next code to check some objects

In [ ]:
classes = []
for i,m in enumerate(dataset_train.image_info):
    info = dataset_train.image_info[i]
    objects = info['objects']
    for j, (objects , pose , truncated , difficult , dims) in enumerate(info['objects']):
        if objects == "None":
            image = dataset_train.load_image(i)
            mask, class_ids = dataset_train.load_mask(i)
            visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)
            print(info['path'])

In [ ]:
model = modellib.MaskRCNN(mode = "training" , config = config , model_dir = MODEL_DIR)

In [ ]:
init_with = "coco"  # imagenet, coco

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name = True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    model.load_weights(COCO_MODEL_PATH , by_name = True, exclude = ["mrcnn_class_logits" , "mrcnn_bbox_fc" , "mrcnn_bbox" , "mrcnn_mask"])

In [ ]:
model.train(dataset_train , dataset_test , learning_rate = config.LEARNING_RATE , epochs = 1, layers = 'all')

# Running model on test data set

In [ ]:
class InferenceConfig(ShapesConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig(num_classes = 6)

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode = "inference", 
                          config = inference_config,
                          model_dir = MODEL_DIR)

# Get path to saved weights
model_path = model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name = True)

In [ ]:
image_id = random.choice(dataset_test.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_test, inference_config, 
                           image_id, use_mini_mask = False)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_train.class_names, figsize = (8, 8))

In [ ]:
results = model.detect([original_image], verbose = 1)

r = results[0]
visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_test.class_names, r['scores'], ax = get_ax())

# Accuracy

In [ ]:
image_ids = np.random.choice(dataset_test.image_ids , 10)
APs = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_test, inference_config,
                               image_id, use_mini_mask = False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)
    
print("mAP: ", np.mean(APs))